<a href="https://colab.research.google.com/github/omkarpat/COVID-Medical-Document-Visual-Aid/blob/main/KG_COVID_19_Data_File_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and preprocess the Dataset into files which can be loaded effectively in the memory

In [2]:
!wget https://kg-hub.berkeleybop.io/kg-covid-19/current/kg-covid-19.tar.gz
!tar -xf kg-covid-19.tar.gz
!ls

--2020-12-10 02:18:27--  https://kg-hub.berkeleybop.io/kg-covid-19/current/kg-covid-19.tar.gz
Resolving kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)... 54.192.86.123, 54.192.86.55, 54.192.86.74, ...
Connecting to kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)|54.192.86.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 376275636 (359M) [application/gzip]
Saving to: ‘kg-covid-19.tar.gz.1’

kg-covid-19.tar.gz. 100%[===================>] 358.84M   171MB/s    in 2.1s    

2020-12-10 02:18:29 (171 MB/s) - ‘kg-covid-19.tar.gz.1’ saved [376275636/376275636]

kg-covid-19.tar.gz    merged-kg_edges.tsv  sample_data
kg-covid-19.tar.gz.1  merged-kg_nodes.tsv


In [3]:
!head -10000000 merged-kg_edges.tsv > merged-kg_edges_head.tsv
!tail -n +10000000 merged-kg_edges.tsv > merged-kg_edges_tail.tsv

## Import packages and set edge columns

In [4]:
import pandas as pd
import json
pd.set_option('display.max_rows', 50)

columns = ['id', 'subject', 'edge_label', 'object', 'relation', 'provided_by', 'target_type', 'standard_units', 'subjectActivity', 'uo_units', 'assay_organism', 'neighborhood', 'assay', 'textmining', 'type', 'ECO_code', 'coexpression', 'cooccurence', 'edge_key', 'coexpression_transferred', 'experiments_transferred', 'standard_value', 'target_organism', 'publication', 'comment', 'Annotation_Properties', 'evidence', 'textmining_transferred', 'neighborhood_transferred', 'fusion', 'database_transferred', 'database', 'subj_exp_role', 'Assigned_by', 'num_participants', 'standard_relation', 'DB_References', 'combined_score', 'obj_exp_role', 'detection_method', 'experiments', 'objectActivity', 'standard_type', 'With', 'Date', 'homology', 'association_type', 'publications', 'target_pref_name']

## Load Nodes and Edges into dataframes

In [5]:
kg_nodes_df = pd.read_csv('merged-kg_nodes.tsv', sep='\t', header=0, dtype={"id":"string",	"name":"string",	"category":"string",	"provided_by":"string",	"TTD_ID":"string",	"iri":"string",	"doi":"string",	"title":"string",	"same_as":"string",	"tissue_chembl_id":"string",	"TDL":"string",	"description":"string",	"in_taxon	assay_type":"string",	"bao_format":"string",	"confidence_score":"string",	"molecular_formula":"string",	"synonym":"string",	"assay_chembl_id":"string",	"canonical_smiles":"string",	"pubmed_id":"string",	"ncbi_taxid":"string",	"bao_label":"string",	"xrefs":"string",	"inorganic_flag":float,	"full_name":"string",	"subsets":"string",	"molecule_type":"string",	"polymer_flag":float,	"natural_product":float})
kg_nodes_df = kg_nodes_df[kg_nodes_df['name'].notna()]
kg_nodes_df.drop(['doi', 'assay_type','title','tissue_chembl_id','TDL','in_taxon','assay_type','bao_format','confidence_score','assay_chembl_id','pubmed_id','bao_label'], axis=1)
kg_nodes_df['name'] = kg_nodes_df['name'].str.lower()
kg_nodes_df['node_type'] = kg_nodes_df.id.str.split(":").str[0]

kg_edges_df = pd.read_csv('merged-kg_edges_head.tsv', sep='\t', header=0)
kg_edges_df = kg_edges_df.drop(['id','neighborhood','textmining','coexpression', 'cooccurence','edge_key','coexpression_transferred','experiments_transferred','standard_value','textmining_transferred','neighborhood_transferred','fusion','database_transferred','database','num_participants','standard_relation','combined_score','experiments','With','Date','publications'], axis=1)
kg_edges_tail_df = pd.read_csv('merged-kg_edges_tail.tsv', sep='\t')
kg_edges_tail_df.columns = columns
kg_edges_tail_df = kg_edges_tail_df.drop(['id','neighborhood','textmining','coexpression', 'cooccurence','edge_key','coexpression_transferred','experiments_transferred','standard_value','textmining_transferred','neighborhood_transferred','fusion','database_transferred','database','num_participants','standard_relation','combined_score','experiments','With','Date','publications'], axis=1)
kg_edges_df = kg_edges_df.append(kg_edges_tail_df)
kg_edges_tail_df = None

print(len(kg_nodes_df), len(kg_edges_df))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,8,23,26,32,38,39,41,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,6,7,8,9,10,12,14,15,18,22,23,24,25,26,32,33,35,36,38,39,41,42,43,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


307818 21443188


In [6]:
kg_nodes_df.head()

,id,name,category,provided_by,TTD_ID,iri,doi,title,same_as,tissue_chembl_id,TDL,description,in_taxon,assay_type,bao_format,confidence_score,molecular_formula,synonym,assay_chembl_id,canonical_smiles,pubmed_id,ncbi_taxid,bao_label,xrefs,inorganic_flag,full_name,subsets,molecule_type,polymer_flag,natural_product,node_type
0,ENSEMBL:ENSG00000004059,arf5,biolink:Gene,STRING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,ADP ribosylation factor 5,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NCBIGene:381,NaN,<NA>,<NA>,<NA>,NaN,NaN,ENSEMBL
1,ENSEMBL:ENSG00000143933,calm2,biolink:Gene,STRING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,calmodulin 2,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NCBIGene:805,NaN,<NA>,<NA>,<NA>,NaN,NaN,ENSEMBL
2,ENSEMBL:ENSG00000131089,arhgef9,biolink:Gene,STRING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cdc42 guanine nucleotide exchange factor 9,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NCBIGene:23229,NaN,<NA>,<NA>,<NA>,NaN,NaN,ENSEMBL
3,ENSEMBL:ENSG00000178607,ern1,biolink:Gene,STRING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,endoplasmic reticulum to nucleus signaling 1,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NCBIGene:2081,NaN,<NA>,<NA>,<NA>,NaN,NaN,ENSEMBL
4,ENSEMBL:ENSG00000147889,cdkn2a,biolink:Gene,STRING,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,cyclin dependent kinase inhibitor 2A,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NCBIGene:1029,NaN,<NA>,<NA>,<NA>,NaN,NaN,ENSEMBL


In [7]:
kg_edges_df.head()

,subject,edge_label,object,relation,provided_by,target_type,standard_units,subjectActivity,uo_units,assay_organism,assay,type,ECO_code,target_organism,publication,comment,Annotation_Properties,evidence,subj_exp_role,Assigned_by,DB_References,obj_exp_role,detection_method,objectActivity,standard_type,homology,association_type,target_pref_name
0,ENSEMBL:ENSG00000004059,biolink:has_gene_product,UniProtKB:P84085,RO:0002205,NCBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSEMBL:ENSG00000143933,biolink:has_gene_product,UniProtKB:P0DP24,RO:0002205,NCBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSEMBL:ENSG00000131089,biolink:has_gene_product,UniProtKB:O43307,RO:0002205,NCBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSEMBL:ENSG00000178607,biolink:has_gene_product,UniProtKB:O75460,RO:0002205,NCBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSEMBL:ENSG00000147889,biolink:has_gene_product,UniProtKB:P42771,RO:0002205,NCBI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Some Helper Functions to generate the data

In [8]:
def next_level_relations(edge_node_list, drug_nodes, drug_edges):

  drug_nodes = kg_nodes_df[kg_nodes_df.id.isin(edge_node_list)]

  subject_edges = kg_edges_df[kg_edges_df["subject"].isin(edge_node_list)]
  object_edges = kg_edges_df[kg_edges_df["object"].isin(edge_node_list)]
  drug_edges = pd.concat([subject_edges, object_edges])
  #subject_tail_edges = kg_edges_tail_df[kg_edges_tail_df["subject"].isin(edge_node_list)]
  #object_tail_edges = kg_edges_tail_df[kg_edges_tail_df["object"].isin(edge_node_list)]
  #drug_edges = pd.concat([subject_edges, object_edges, subject_tail_edges, object_tail_edges])

  return drug_nodes, drug_edges


def create_data_json(term, depth=0, generate_flag=False):
  drug_nodes = kg_nodes_df[kg_nodes_df.name.str.contains(term.lower())]
  node_list = list(drug_nodes.id)

  subject_edges = kg_edges_df[kg_edges_df["subject"].isin(node_list)]
  object_edges = kg_edges_df[kg_edges_df["object"].isin(node_list)]
  drug_edges = pd.concat([subject_edges, object_edges])
  #subject_tail_edges = kg_edges_tail_df[kg_edges_tail_df["subject"].isin(node_list)]
  #object_tail_edges = kg_edges_tail_df[kg_edges_tail_df["object"].isin(node_list)]
  #drug_edges = pd.concat([subject_edges, object_edges, subject_tail_edges, object_tail_edges])

  for depth_iter in range(depth):
    edge_node_list = list(set(list(drug_edges.subject) + list(drug_edges.object)))
    drug_nodes, drug_edges = next_level_relations(edge_node_list, drug_nodes, drug_edges)

  edge_node_list = list(set(list(drug_edges.subject) + list(drug_edges.object)))
  drug_nodes = kg_nodes_df[kg_nodes_df.id.isin(edge_node_list)]

  #drug_edges = drug_edges[drug_edges.publication != ""]
  
  links_df = drug_edges[['subject','edge_label','object','relation','provided_by','association_type','publication','ECO_code']]
  links_df.columns = ['source', 'value', 'target','relation','provided_by','association_type','publication','ECO_code']
  edges_df = drug_nodes[["id","name","category","provided_by"]]
  edges_df.columns = ['id', 'name', 'group', 'provided_by']

  data = {}
  data["nodes"] = json.loads(edges_df.to_json(orient="records"))
  data["links"] = json.loads(links_df.to_json(orient="records"))

  node_ids = [data["nodes"][i]["id"] for i in range(len(data["nodes"]))]

  trimmed_links = []
  for i in range(len(data["links"])):
    if data["links"][i]["source"] in node_ids and data["links"][i]["target"] in node_ids:
      trimmed_links.append(data["links"][i])
  data["links"] = trimmed_links

  if generate_flag or len(data["links"]) > 100:
    with open( term + ".json", "w") as outfile:
      json.dump(data, outfile, indent=4)
  return len(data["nodes"]), len(data["links"])

def generate_bubble_chart_data(data_type, column):
  if data_type == "nodes":
    data_df = kg_nodes_df[column].value_counts().to_frame('counts')
  else:
    data_df = kg_edges_df[column].value_counts().to_frame('counts')
    #head_data_df = kg_edges_df[column].value_counts().to_frame('counts')
    #tail_data_df = kg_edges_tail_df[column].value_counts().to_frame('counts')
    #data_df = pd.concat([head_data_df, tail_data_df])
  data_df[column] = list(data_df.index)
  data_df = pd.concat([pd.Series(row['counts'], row[column].split('|')) for _, row in data_df.iterrows()]).reset_index()
  data_df.columns = [column,"counts"]
  data_df = data_df.groupby(column).sum()
  data_df[column] = list(data_df.index)
  data_df.reset_index(drop=True, inplace=True)
  data_df.columns = ["value","id"]
  data_df.to_csv("%s_%s.csv" % (data_type, column))

## Samples

In [ ]:
create_data_json("ubql1_human", generate_flag=True)

In [ ]:
generate_bubble_chart_data("nodes","node_type")

In [9]:
!ls -lh

total 8.3G
-rw-r--r-- 1 root root 359M Dec  3 22:19 kg-covid-19.tar.gz
-rw-r--r-- 1 root root 359M Dec  3 22:19 kg-covid-19.tar.gz.1
-rw-r--r-- 1 root root 2.0G Dec 10 02:19 merged-kg_edges_head.tsv
-rw-r--r-- 1 root root 1.8G Dec 10 02:19 merged-kg_edges_tail.tsv
-rw-r--r-- 1  114  120 3.8G Dec  3 03:22 merged-kg_edges.tsv
-rw-r--r-- 1  114  120  83M Dec  3 03:01 merged-kg_nodes.tsv
drwxr-xr-x 1 root root 4.0K Dec  2 22:04 sample_data


In [12]:
print(len(set(list(kg_edges_df.subject) + list(kg_edges_df.object))), len(list(kg_nodes_df.id)))

372799 307818
